# Melting Point Prediction - Enhanced Features

## 목표
- train_fully.csv (20,182개)로 학습
- train.csv (2,662개)로 검증

## 1. Setup & Configuration

In [1]:
# Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import pickle
from pathlib import Path

# RDKit
from rdkit import Chem, RDLogger
from rdkit.Chem import Descriptors, rdMolDescriptors, AllChem, Fragments, Crippen, Lipinski
from rdkit.Chem.AllChem import GetMorganFingerprintAsBitVect, ComputeGasteigerCharges
from rdkit.Chem.MACCSkeys import GenMACCSKeys
from rdkit.Chem.EState import EState_VSA, AtomTypes as EAtomTypes
from rdkit.Chem.Scaffolds import MurckoScaffold
from rdkit.Avalon import pyAvalonTools

# ML Models
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import mean_absolute_error

# Optimization
from joblib import Parallel, delayed
from tqdm import tqdm

# Configuration
SEED = 42
N_JOBS_FEATURE = 40
N_JOBS_MODEL = 20
DATA_PATH = Path('/home/yoo122333/project/saja/data')
FEATURE_PATH = Path('/home/yoo122333/project/saja/features')
FEATURE_PATH.mkdir(exist_ok=True)

# Suppress warnings
warnings.filterwarnings('ignore')
RDLogger.DisableLog('rdApp.*')

print(f"Setup complete!")
print(f"Seed: {SEED}")
print(f"N_JOBS - Feature: {N_JOBS_FEATURE}, Model: {N_JOBS_MODEL}")

Setup complete!
Seed: 42
N_JOBS - Feature: 40, Model: 20


## 2. Data Loading

In [2]:
# Train: train_fully.csv
df_train = pd.read_csv(DATA_PATH / 'train_fully.csv')
df_train = df_train[['SMILES', 'Tm']].dropna().reset_index(drop=True)

# Val: train.csv
df_val = pd.read_csv(DATA_PATH / 'train.csv')
df_val = df_val[['id', 'SMILES', 'Tm']].dropna().reset_index(drop=True)

# Test: test.csv + submission_v6.csv
df_test = pd.read_csv(DATA_PATH / 'test.csv')
df_submission = pd.read_csv(DATA_PATH / 'submission_v6.csv')
df_test = df_test[['id', 'SMILES']].merge(df_submission, on='id', how='left')

print(f"Train: {len(df_train)} samples")
print(f"Val:   {len(df_val)} samples")
print(f"Test:  {len(df_test)} samples")

Train: 20182 samples
Val:   2662 samples
Test:  666 samples


## 3. Feature Engineering Functions

In [3]:
# Helper functions
def _safe(f, default=None):
    def wrap(*args, **kwargs):
        try:
            return f(*args, **kwargs)
        except Exception:
            return default
    return wrap

def drop_constant_and_duplicate_columns(df):
    nunique = df.nunique(dropna=False)
    constant_cols = nunique[nunique <= 1].index.tolist()
    df = df.drop(columns=constant_cols)
    df = df.loc[:, ~df.T.duplicated(keep='first')]
    return df

def gasteiger_stats(m):
    m = Chem.AddHs(m)
    ComputeGasteigerCharges(m)
    vals = []
    for a in m.GetAtoms():
        v = a.GetDoubleProp('_GasteigerCharge') if a.HasProp('_GasteigerCharge') else 0.0
        if pd.isna(v) or v == float('inf') or v == float('-inf'):
            v = 0.0
        vals.append(v)
    arr = np.array(vals, dtype=float)
    return {
        "Gasteiger_q_sum": float(arr.sum()),
        "Gasteiger_q_abs_sum": float(np.abs(arr).sum()),
        "Gasteiger_q_min": float(arr.min(initial=0.0)),
        "Gasteiger_q_max": float(arr.max(initial=0.0)),
        "Gasteiger_q_std": float(arr.std(ddof=0)),
    }

def _bond_order(b):
    if b.GetIsAromatic():
        return 1.5
    t = b.GetBondType()
    if t == Chem.BondType.SINGLE: return 1
    if t == Chem.BondType.DOUBLE: return 2
    if t == Chem.BondType.TRIPLE: return 3
    return 0

def _ring_size_hist(m):
    ri = m.GetRingInfo()
    sizes = [len(r) for r in ri.AtomRings()]
    out = {5:0, 6:0, 7:0, 8:0}
    for s in sizes:
        if s in out: out[s] += 1
    return out, len(sizes)

def _ring_systems_count(m):
    ri = m.GetRingInfo()
    rings = [set(r) for r in ri.AtomRings()]
    if not rings: return 0
    seen = set()
    sys = 0
    for i in range(len(rings)):
        if i in seen: continue
        sys += 1
        stack = [i]
        seen.add(i)
        while stack:
            j = stack.pop()
            for k in range(len(rings)):
                if k in seen: continue
                if rings[j] & rings[k]:
                    seen.add(k); stack.append(k)
    return sys

def _murcko_stats(m):
    try:
        scaf = MurckoScaffold.GetScaffoldForMol(m)
        if scaf is None or scaf.GetNumAtoms() == 0:
            return {"MurckoAtoms":0, "MurckoRings":0, "MurckoRingSystems":0, "SideChainAtoms":m.GetNumAtoms()}
        msys = _ring_systems_count(scaf)
        return {
            "MurckoAtoms": scaf.GetNumAtoms(),
            "MurckoRings": rdMolDescriptors.CalcNumRings(scaf),
            "MurckoRingSystems": msys,
            "SideChainAtoms": m.GetNumAtoms() - scaf.GetNumAtoms(),
        }
    except:
        return {"MurckoAtoms":0, "MurckoRings":0, "MurckoRingSystems":0, "SideChainAtoms":m.GetNumAtoms()}

def _estate_stats(m):
    try:
        vals = EAtomTypes.EStateIndices(m)
        if not vals: return {"EState_sum":0.0,"EState_mean":0.0,"EState_max":0.0,"EState_min":0.0,"EState_std":0.0}
        arr = np.asarray(vals, dtype=float)
        return {
            "EState_sum": float(arr.sum()),
            "EState_mean": float(arr.mean()),
            "EState_max": float(arr.max()),
            "EState_min": float(arr.min()),
            "EState_std": float(arr.std(ddof=0)),
        }
    except:
        return {"EState_sum":0.0,"EState_mean":0.0,"EState_max":0.0,"EState_min":0.0,"EState_std":0.0}

def _smiles_morphology(smi: str):
    if not smi: 
        return {"SMI_len":0,"SMI_branches":0,"SMI_ringDigits":0,"SMI_stereoAt":0,"SMI_ezSlashes":0}
    return {
        "SMI_len": len(smi),
        "SMI_branches": smi.count("("),
        "SMI_ringDigits": sum(ch.isdigit() for ch in smi),
        "SMI_stereoAt": smi.count("@"),
        "SMI_ezSlashes": smi.count("/") + smi.count("\\"),
    }

def augment_extra_cheaps(m, row):
    bonds = m.GetBonds()
    if bonds:
        single = sum(1 for b in bonds if b.GetBondType() == Chem.BondType.SINGLE)
        double = sum(1 for b in bonds if b.GetBondType() == Chem.BondType.DOUBLE)
        triple = sum(1 for b in bonds if b.GetBondType() == Chem.BondType.TRIPLE)
        arom = sum(1 for b in bonds if b.GetIsAromatic())
        nb = len(bonds)
        row["FracSingle"] = single / nb if nb else 0.0
        row["FracDouble"] = double / nb if nb else 0.0
        row["FracAromatic"] = arom / nb if nb else 0.0
        orders = [_bond_order(b) for b in bonds]
        row["MeanBondOrder"] = np.mean(orders) if orders else 0.0
    else:
        row["FracSingle"] = row["FracDouble"] = row["FracAromatic"] = row["MeanBondOrder"] = 0.0
    
    hist, total_rings = _ring_size_hist(m)
    row["Ring5"] = hist[5]
    row["Ring6"] = hist[6]
    row["Ring7"] = hist[7]
    row["Ring8"] = hist[8]
    row["TotalRings"] = total_rings
    row["RingSystems"] = _ring_systems_count(m)
    
    murcko = _murcko_stats(m)
    row.update(murcko)
    
    gast = gasteiger_stats(m)
    row.update(gast)
    
    return row

print("Helper functions defined")

Helper functions defined


In [4]:
# 3D Shape features
def _shape3d_worker(cansmi: str, maxIters: int = 0):
    try:
        m = Chem.MolFromSmiles(cansmi)
        if m is None:
            return cansmi, {}

        mH = Chem.AddHs(m)
        params = AllChem.ETKDGv3()
        params.randomSeed = 123
        params.useRandomCoords = True

        with Chem.WrapLogs():
            cid = AllChem.EmbedMolecule(mH, params)
        if cid < 0:
            with Chem.WrapLogs():
                cid = AllChem.EmbedMolecule(mH, randomSeed=123)
            if cid < 0:
                return cansmi, {}

        if maxIters and maxIters > 0:
            try:
                with Chem.WrapLogs():
                    AllChem.UFFOptimizeMolecule(mH, confId=cid, maxIters=int(maxIters))
            except:
                pass

        m_noH = Chem.RemoveHs(mH)
        confId = 0

        out = {}
        for nm, fn in [
            ("RadiusOfGyration", rdMolDescriptors.CalcRadiusOfGyration),
            ("InertialShapeFactor", rdMolDescriptors.CalcInertialShapeFactor),
            ("PMI1", rdMolDescriptors.CalcPMI1),
            ("PMI2", rdMolDescriptors.CalcPMI2),
            ("PMI3", rdMolDescriptors.CalcPMI3),
            ("NPR1", rdMolDescriptors.CalcNPR1),
            ("NPR2", rdMolDescriptors.CalcNPR2),
        ]:
            try:
                with Chem.WrapLogs():
                    out[nm] = float(fn(m_noH, confId=confId))
            except:
                out[nm] = 0.0

        pmi1 = out.get("PMI1", 0.0) or 0.0
        pmi2 = out.get("PMI2", 0.0) or 0.0
        pmi3 = out.get("PMI3", 0.0) or 0.0
        out["PMI2_over_PMI1"] = (pmi2 / pmi1) if pmi1 else 0.0
        out["PMI3_over_PMI1"] = (pmi3 / pmi1) if pmi1 else 0.0

        return cansmi, out
    except:
        return cansmi, {}

def precompute_shape3d_cache(smiles_series, n_jobs=40, maxIters=0):
    print(f"Precomputing 3D shape cache with {n_jobs} cores...")
    
    can = smiles_series.astype(str).apply(lambda s: Chem.MolToSmiles(Chem.MolFromSmiles(s), canonical=True)
                                          if pd.notna(s) and Chem.MolFromSmiles(s) is not None else None)
    uniq = sorted(x for x in set(can.tolist()) if x is not None)

    if not uniq:
        return {}

    print(f"Computing 3D features for {len(uniq)} unique molecules...")
    results = Parallel(n_jobs=n_jobs, backend="loky", batch_size=64, verbose=10)(
        delayed(_shape3d_worker)(s, maxIters) for s in uniq
    )
    
    cache = {k: v for k, v in results if k is not None}
    print(f"3D cache complete: {len(cache)} molecules")
    return cache

print("3D shape functions defined")

3D shape functions defined


In [5]:
# Main feature extraction function
def rdkit_feature_row(smiles_str, shape3d_cache=None):
    """
    Extract ~7000+ features:
    - RDKit 2D descriptors (~210)
    - Morgan FP radius 1,2,3 (3072 bits)
    - MACCS Keys (167 bits)
    - Atom Pair FP (1024 bits)
    - Topological Torsion (1024 bits)
    - RDKit FP (2048 bits)
    - Avalon FP (1024 bits)
    - Element counts, fragments, VSA, EState
    - Melting point specific features
    - 3D geometry
    """
    row = {}
    
    try:
        m = Chem.MolFromSmiles(smiles_str)
        if m is None:
            return None
        
        cansmi = Chem.MolToSmiles(m, canonical=True)
        
        # 1. RDKit 2D Descriptors
        for name, func in Descriptors._descList:
            row[name] = _safe(func, 0.0)(m)
        
        # 2. Basic descriptors
        row['MolLogP'] = Crippen.MolLogP(m)
        row['TPSA'] = rdMolDescriptors.CalcTPSA(m)
        row['NumHDonors'] = Lipinski.NumHDonors(m)
        row['NumHAcceptors'] = Lipinski.NumHAcceptors(m)
        row['NumRings'] = rdMolDescriptors.CalcNumRings(m)
        row['NumAromaticRings'] = rdMolDescriptors.CalcNumAromaticRings(m)
        row['NumSaturatedRings'] = rdMolDescriptors.CalcNumSaturatedRings(m)
        row['NumAliphaticRings'] = rdMolDescriptors.CalcNumAliphaticRings(m)
        row['NumHeteroatoms'] = rdMolDescriptors.CalcNumHeteroatoms(m)
        row['NumRotatableBonds'] = rdMolDescriptors.CalcNumRotatableBonds(m)
        
        # 3. Element counts
        for el in ['C', 'H', 'N', 'O', 'S', 'F', 'Cl', 'Br', 'I', 'P']:
            row[f'Count_{el}'] = sum(1 for a in m.GetAtoms() if a.GetSymbol() == el)
        
        # 4. Fragment counts
        for attr in dir(Fragments):
            if attr.startswith('fr_'):
                row[attr] = _safe(getattr(Fragments, attr), 0)(m)
        
        # 5. EXTENDED FINGERPRINTS
        # Morgan radius 1, 2, 3
        for radius in [1, 2, 3]:
            morgan = GetMorganFingerprintAsBitVect(m, radius=radius, nBits=1024)
            for i in range(1024):
                row[f'Morgan_r{radius}_{i}'] = morgan[i]
        
        # MACCS Keys
        maccs = GenMACCSKeys(m)
        for i in range(167):
            row[f'MACCS_{i}'] = maccs[i]
        
        # Atom Pair
        try:
            ap = rdMolDescriptors.GetHashedAtomPairFingerprintAsBitVect(m, nBits=1024)
            for i in range(1024):
                row[f'AtomPair_{i}'] = ap[i]
        except:
            for i in range(1024):
                row[f'AtomPair_{i}'] = 0
        
        # Topological Torsion
        try:
            tt = rdMolDescriptors.GetHashedTopologicalTorsionFingerprintAsBitVect(m, nBits=1024)
            for i in range(1024):
                row[f'TopTorsion_{i}'] = tt[i]
        except:
            for i in range(1024):
                row[f'TopTorsion_{i}'] = 0
        
        # RDKit Fingerprint
        try:
            rdk = Chem.RDKFingerprint(m, fpSize=2048)
            for i in range(2048):
                row[f'RDKitFP_{i}'] = rdk[i]
        except:
            for i in range(2048):
                row[f'RDKitFP_{i}'] = 0
        
        # Avalon Fingerprint
        try:
            avalon = pyAvalonTools.GetAvalonFP(m, nBits=1024)
            for i in range(1024):
                row[f'Avalon_{i}'] = avalon[i]
        except:
            for i in range(1024):
                row[f'Avalon_{i}'] = 0
        
        # 6. VSA binnings
        try:
            slogp_vsa = rdMolDescriptors.SlogP_VSA_(m)
            for i, v in enumerate(slogp_vsa):
                row[f'SlogP_VSA{i}'] = v
        except:
            pass
        
        try:
            smr_vsa = rdMolDescriptors.SMR_VSA_(m)
            for i, v in enumerate(smr_vsa):
                row[f'SMR_VSA{i}'] = v
        except:
            pass
        
        try:
            peoe_vsa = rdMolDescriptors.PEOE_VSA_(m)
            for i, v in enumerate(peoe_vsa):
                row[f'PEOE_VSA{i}'] = v
        except:
            pass
        
        # 7. EState VSA
        try:
            estate_vsa = EState_VSA.EState_VSA_(m)
            for i, v in enumerate(estate_vsa):
                row[f'EState_VSA{i}'] = v
        except:
            pass
        
        # 8. EState statistics
        estate = _estate_stats(m)
        row.update(estate)
        
        # 9. Extra features
        row = augment_extra_cheaps(m, row)
        
        # 10. SMILES morphology
        smi_morph = _smiles_morphology(smiles_str)
        row.update(smi_morph)
        
        # 11. MELTING POINT SPECIFIC FEATURES
        natoms = m.GetNumAtoms()
        
        # Molecular symmetry
        try:
            ranks = Chem.CanonicalRankAtoms(m, breakTies=False)
            row['NumSymmetryClasses'] = len(set(ranks))
            row['SymmetryRatio'] = len(set(ranks)) / natoms if natoms > 0 else 0
        except:
            row['NumSymmetryClasses'] = natoms
            row['SymmetryRatio'] = 1.0
        
        # Surface area
        try:
            row['LabuteASA'] = rdMolDescriptors.CalcLabuteASA(m)
        except:
            row['LabuteASA'] = 0.0
        
        # Partial charges
        try:
            row['MaxPartialCharge'] = Descriptors.MaxPartialCharge(m) or 0.0
            row['MinPartialCharge'] = Descriptors.MinPartialCharge(m) or 0.0
            row['PartialChargeRange'] = row['MaxPartialCharge'] - row['MinPartialCharge']
        except:
            row['MaxPartialCharge'] = 0.0
            row['MinPartialCharge'] = 0.0
            row['PartialChargeRange'] = 0.0
        
        # Aromatic fraction
        aromatic_atoms = sum(1 for a in m.GetAtoms() if a.GetIsAromatic())
        row['AromaticAtomFrac'] = aromatic_atoms / natoms if natoms > 0 else 0
        
        # Heteroatom types
        hetero_atoms = [a for a in m.GetAtoms() if a.GetSymbol() not in ['C', 'H']]
        row['NumHeteroAtomTypes'] = len(set(a.GetSymbol() for a in hetero_atoms))
        
        # sp3 fraction
        try:
            row['FractionCSP3'] = rdMolDescriptors.CalcFractionCSP3(m)
        except:
            row['FractionCSP3'] = 0.0
        
        # Hall-Kier Alpha
        try:
            row['HallKierAlpha'] = Descriptors.HallKierAlpha(m)
        except:
            row['HallKierAlpha'] = 0.0
        
        # 12. CUSTOM INTERACTION FEATURES
        logp = row.get('MolLogP', 0.0)
        tpsa = row.get('TPSA', 1.0)
        mw = row.get('MolWt', 1.0)
        hbd = row.get('NumHDonors', 0.0)
        hba = row.get('NumHAcceptors', 0.0)
        nrot = row.get('NumRotatableBonds', 0.0)
        nring = row.get('NumRings', 0.0)
        narom = row.get('NumAromaticRings', 0.0)
        hetero = row.get('NumHeteroatoms', 0.0)
        
        row['HBondCapacity'] = hbd + hba
        row['HBondDensity'] = (hbd + hba) / natoms if natoms > 0 else 0.0
        row['HBond_Product'] = hbd * hba
        row['Rigidity_Score'] = (nring / natoms) if natoms > 0 else 0.0
        row['Flexibility_Score'] = (nrot / natoms) if natoms > 0 else 0.0
        row['AromRingFrac'] = (narom / nring) if nring > 0 else 0.0
        row['LogP_div_TPSA'] = (logp / tpsa) if tpsa > 0 else 0.0
        row['Complexity_per_MW'] = (natoms / mw) if mw > 0 else 0.0
        row['HeteroAtomFrac'] = (hetero / natoms) if natoms > 0 else 0.0
        row['MW_per_Ring'] = mw / nring if nring > 0 else mw
        row['MW_per_RotBond'] = mw / (nrot + 1)
        row['LogP_times_MW'] = logp * mw
        row['TPSA_per_MW'] = tpsa / mw if mw > 0 else 0.0
        
        # 13. 3D Shape
        if shape3d_cache and cansmi in shape3d_cache:
            row.update(shape3d_cache[cansmi])
        else:
            for k in ["RadiusOfGyration", "InertialShapeFactor", 
                     "PMI1", "PMI2", "PMI3", "NPR1", "NPR2",
                     "PMI2_over_PMI1", "PMI3_over_PMI1"]:
                row[k] = 0.0
        
        return row
        
    except:
        return None

print("rdkit_feature_row() defined - ~7000+ features")

rdkit_feature_row() defined - ~7000+ features


In [6]:
# Featurization wrapper
def featurize_smiles(df, smiles_col='SMILES_can', shape3d_cache=None):
    print(f"Extracting features from {len(df)} molecules...")
    
    rows = []
    for idx, smi in enumerate(tqdm(df[smiles_col], desc="Features")):
        if pd.isna(smi):
            rows.append(None)
            continue
        feat = rdkit_feature_row(smi, shape3d_cache=shape3d_cache)
        rows.append(feat)
    
    df_feat = pd.DataFrame(rows)
    print(f"Initial features: {df_feat.shape[1]} columns")
    
    df_feat.replace([np.inf, -np.inf], np.nan, inplace=True)
    df_feat.fillna(0, inplace=True)
    
    print("Removing constant and duplicate columns...")
    df_feat = drop_constant_and_duplicate_columns(df_feat)
    print(f"After cleanup: {df_feat.shape[1]} columns")
    
    # Reduce memory
    for col in df_feat.select_dtypes(include=['float64']).columns:
        df_feat[col] = df_feat[col].astype('float32')
    
    print(f"Final shape: {df_feat.shape}")
    print(f"Memory: {df_feat.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
    
    return df_feat

print("featurize_smiles() defined")

featurize_smiles() defined


## 4. Feature Extraction & Caching

In [7]:
# Canonical SMILES
def canonicalize(smiles):
    try:
        mol = Chem.MolFromSmiles(smiles)
        return Chem.MolToSmiles(mol, canonical=True) if mol else None
    except:
        return None

df_train['SMILES_can'] = df_train['SMILES'].apply(canonicalize)
df_val['SMILES_can'] = df_val['SMILES'].apply(canonicalize)
df_test['SMILES_can'] = df_test['SMILES'].apply(canonicalize)

df_train = df_train[df_train['SMILES_can'].notna()].reset_index(drop=True)
df_val = df_val[df_val['SMILES_can'].notna()].reset_index(drop=True)
df_test = df_test[df_test['SMILES_can'].notna()].reset_index(drop=True)

print(f"Train: {len(df_train)}, Val: {len(df_val)}, Test: {len(df_test)}")

Train: 20182, Val: 2662, Test: 666


In [8]:
# 3D geometry cache
shape3d_cache_path = FEATURE_PATH / 'shape3d_cache_enhanced.pkl'

if shape3d_cache_path.exists():
    print("Loading 3D cache...")
    with open(shape3d_cache_path, 'rb') as f:
        shape3d_cache = pickle.load(f)
    print(f"Loaded {len(shape3d_cache)} 3D geometries")
else:
    all_smiles = pd.concat([df_train['SMILES_can'], df_val['SMILES_can'], df_test['SMILES_can']]).dropna()
    shape3d_cache = precompute_shape3d_cache(all_smiles, n_jobs=N_JOBS_FEATURE, maxIters=0)
    
    with open(shape3d_cache_path, 'wb') as f:
        pickle.dump(shape3d_cache, f)
    print(f"Saved 3D cache: {len(shape3d_cache)} molecules")

Precomputing 3D shape cache with 40 cores...
Computing 3D features for 20282 unique molecules...


[Parallel(n_jobs=40)]: Using backend LokyBackend with 40 concurrent workers.
[Parallel(n_jobs=40)]: Done   2 tasks      | elapsed:    2.6s
[Parallel(n_jobs=40)]: Done  20 tasks      | elapsed:    2.7s
[Parallel(n_jobs=40)]: Done  42 tasks      | elapsed:    2.8s
[Parallel(n_jobs=40)]: Done  64 tasks      | elapsed:    2.8s
[Parallel(n_jobs=40)]: Done 648 tasks      | elapsed:    2.9s
[Parallel(n_jobs=40)]: Done 1232 tasks      | elapsed:    3.0s
[Parallel(n_jobs=40)]: Done 2192 tasks      | elapsed:    3.1s
[Parallel(n_jobs=40)]: Done 3152 tasks      | elapsed:    3.1s
[Parallel(n_jobs=40)]: Done 4240 tasks      | elapsed:    3.2s
[Parallel(n_jobs=40)]: Done 5328 tasks      | elapsed:    6.0s
[Parallel(n_jobs=40)]: Done 6544 tasks      | elapsed:    6.1s
[Parallel(n_jobs=40)]: Done 7760 tasks      | elapsed:    6.3s
[Parallel(n_jobs=40)]: Done 9104 tasks      | elapsed:    6.4s
[Parallel(n_jobs=40)]: Done 10448 tasks      | elapsed:    6.7s
[Parallel(n_jobs=40)]: Done 11920 tasks      

3D cache complete: 20282 molecules
Saved 3D cache: 20282 molecules


[Parallel(n_jobs=40)]: Done 20282 out of 20282 | elapsed:   10.8s finished


In [9]:
# Extract Train features
train_feat_path = FEATURE_PATH / 'train_fully_enhanced.pkl'

if train_feat_path.exists():
    print("Loading Train features...")
    df_train_feat = pd.read_pickle(train_feat_path)
    print(f"Loaded: {df_train_feat.shape}")
else:
    df_train_feat = featurize_smiles(df_train, smiles_col='SMILES_can', shape3d_cache=shape3d_cache)
    df_train_feat.to_pickle(train_feat_path)
    print(f"Saved Train features: {df_train_feat.shape}")

df_train_feat['Tm'] = df_train['Tm'].values

Extracting features from 20182 molecules...


Features: 100%|██████████| 20182/20182 [1:02:35<00:00,  5.37it/s]


Initial features: 8638 columns
Removing constant and duplicate columns...
After cleanup: 8520 columns
Final shape: (20182, 8520)
Memory: 1302.02 MB
Saved Train features: (20182, 8520)


In [10]:
# Extract Val features
val_feat_path = FEATURE_PATH / 'val_enhanced.pkl'

if val_feat_path.exists():
    print("Loading Val features...")
    df_val_feat = pd.read_pickle(val_feat_path)
    print(f"Loaded: {df_val_feat.shape}")
else:
    df_val_feat = featurize_smiles(df_val, smiles_col='SMILES_can', shape3d_cache=shape3d_cache)
    df_val_feat.to_pickle(val_feat_path)
    print(f"Saved Val features: {df_val_feat.shape}")

df_val_feat['id'] = df_val['id'].values
df_val_feat['Tm'] = df_val['Tm'].values

Extracting features from 2662 molecules...


Features: 100%|██████████| 2662/2662 [06:46<00:00,  6.55it/s]


Initial features: 8638 columns
Removing constant and duplicate columns...
After cleanup: 7779 columns
Final shape: (2662, 7779)
Memory: 156.71 MB
Saved Val features: (2662, 7779)


In [11]:
# Extract Test features
test_feat_path = FEATURE_PATH / 'test_enhanced.pkl'

if test_feat_path.exists():
    print("Loading Test features...")
    df_test_feat = pd.read_pickle(test_feat_path)
    print(f"Loaded: {df_test_feat.shape}")
else:
    df_test_feat = featurize_smiles(df_test, smiles_col='SMILES_can', shape3d_cache=shape3d_cache)
    df_test_feat.to_pickle(test_feat_path)
    print(f"Saved Test features: {df_test_feat.shape}")

df_test_feat['id'] = df_test['id'].values
df_test_feat['Tm'] = df_test['Tm'].values

Extracting features from 666 molecules...


Features: 100%|██████████| 666/666 [01:36<00:00,  6.90it/s]


Initial features: 8638 columns
Removing constant and duplicate columns...
After cleanup: 6531 columns
Final shape: (666, 6531)
Memory: 32.87 MB
Saved Test features: (666, 6531)


## 5. Data Preparation

In [12]:
# Find common features
feature_cols = [c for c in df_train_feat.columns if c not in ['Tm', 'id']]
val_feature_cols = [c for c in df_val_feat.columns if c not in ['Tm', 'id']]
test_feature_cols = [c for c in df_test_feat.columns if c not in ['Tm', 'id']]

common_features = sorted(set(feature_cols) & set(val_feature_cols) & set(test_feature_cols))
print(f"Common features: {len(common_features)}")

X_train = df_train_feat[common_features].values
y_train = df_train_feat['Tm'].values

X_val = df_val_feat[common_features].values
y_val = df_val_feat['Tm'].values

X_test = df_test_feat[common_features].values
y_test = df_test_feat['Tm'].values

print(f"Train: {X_train.shape}")
print(f"Val:   {X_val.shape}")
print(f"Test:  {X_test.shape}")

Common features: 6476
Train: (20182, 6476)
Val:   (2662, 6476)
Test:  (666, 6476)


## 6. Model Training

In [21]:
print("="*60)
print("Training ExtraTreesRegressor")
print(f"Train: {len(X_train)}, Val: {len(X_val)}")
print("="*60)

# Train model
model = ExtraTreesRegressor(
    n_estimators=100,
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    max_features=None,
    bootstrap=False,
    n_jobs=N_JOBS_MODEL,
    random_state=SEED,
    verbose=1
)

print("\nTraining model...")
model.fit(X_train, y_train)

# Save model
model_path = FEATURE_PATH / 'extratrees_model.pkl'
with open(model_path, 'wb') as f:
    pickle.dump(model, f)
print(f"Model saved: {model_path}")

# Save feature names
feature_info = {
    'common_features': common_features,
    'n_features': len(common_features)
}
feature_info_path = FEATURE_PATH / 'feature_info.pkl'
with open(feature_info_path, 'wb') as f:
    pickle.dump(feature_info, f)
print(f"Feature info saved: {feature_info_path}")

# Predictions
train_pred = model.predict(X_train)
val_pred = model.predict(X_val)

# Evaluate
train_mae = mean_absolute_error(y_train, train_pred)
val_mae = mean_absolute_error(y_val, val_pred)

print("\n" + "="*60)
print("Results")
print("="*60)
print(f"Train MAE: {train_mae:.4f}K")
print(f"Val MAE:   {val_mae:.4f}K")

Training ExtraTreesRegressor
Train: 20182, Val: 2662

Training model...


[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:  2.0min
[Parallel(n_jobs=20)]: Done 100 out of 100 | elapsed: 12.2min finished


Model saved: /home/yoo122333/project/saja/features/extratrees_model.pkl
Feature info saved: /home/yoo122333/project/saja/features/feature_info.pkl


[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.3s
[Parallel(n_jobs=20)]: Done 100 out of 100 | elapsed:    0.8s finished



Results
Train MAE: 0.0155K
Val MAE:   6.7154K


[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.0s
[Parallel(n_jobs=20)]: Done 100 out of 100 | elapsed:    0.2s finished


In [17]:
# Save trained model
model_path = FEATURE_PATH / 'extratrees_model.pkl'
with open(model_path, 'wb') as f:
    pickle.dump(model, f)
print(f"✓ Model saved: {model_path}")

# Save feature names
feature_info = {
    'common_features': common_features,
    'n_features': len(common_features)
}
feature_info_path = FEATURE_PATH / 'feature_info.pkl'
with open(feature_info_path, 'wb') as f:
    pickle.dump(feature_info, f)
print(f"✓ Feature info saved: {feature_info_path}")

# Check file size
import os
model_size = os.path.getsize(model_path) / (1024**2)
print(f"\nModel file size: {model_size:.2f} MB")

✓ Model saved: /home/yoo122333/project/saja/features/extratrees_model.pkl
✓ Feature info saved: /home/yoo122333/project/saja/features/feature_info.pkl

Model file size: 271.12 MB


In [18]:
df_submit = pd.DataFrame({
    'id': df_test['id'].values,
    'Tm': test_pred
})

submit_path = DATA_PATH / 'submission_enhanced_features.csv'
df_submit.to_csv(submit_path, index=False)

print(f"Saved: {submit_path}")
print(f"Samples: {len(df_submit)}")
print(f"\nPrediction stats:")
print(f"  Mean: {df_submit['Tm'].mean():.2f}K")
print(f"  Std:  {df_submit['Tm'].std():.2f}K")
print(f"  Min:  {df_submit['Tm'].min():.2f}K")
print(f"  Max:  {df_submit['Tm'].max():.2f}K")

df_submit.head(10)

Saved: /home/yoo122333/project/saja/data/submission_enhanced_features.csv
Samples: 666

Prediction stats:
  Mean: 271.31K
  Std:  77.64K
  Min:  89.55K
  Max:  562.65K


,id,Tm
0,1022,390.15
1,1146,341.15
2,79,185.25
3,2279,206.85
4,1342,231.45
5,2082,337.85
6,29,198.15
7,515,316.65
8,2309,287.65
9,1177,228.15
